In [ ]:
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import helper_func
import math
import models
import scipy.special as sp
from scipy.misc import logsumexp
import gc

In [ ]:
full_X,x,test_mask = helper_func.load_data()
dataset = 'bibtex' ## name of folder/dataset
metric = 'ndcg'
users = x.shape[0]
items = x.shape[1]
train_non_zero_indices = helper_func.non_zero_entries(x)
score = []

In [ ]:
epochs = 10000
epochs += 1
test_every = 1000
no_samples = 1000
k = 50
param = models.poisson_response();  # 'ztp' or 'normal'

In [ ]:
varpi = 0.1 #looks like 'w^bar' or omega bar
sparsity = 1.0 - float(len(train_non_zero_indices))/(users*items)
em = -np.log(sparsity)
emsq = np.sqrt(em/k)
varrho = 0.1 # looks like mirror inverted q
eta = varrho * emsq  #looks like n
rho =  varrho * varrho  #looks like p
zeta = varpi *emsq #looks like mirror inverted c
omega = varpi * varpi #looks like w
xi = 0.7
tau = 10000.0
cutoff_loglik = np.log(1e-6);
n_trunc = 20;

In [ ]:
t_user = np.ones(shape=users)*tau
t_item = np.ones(shape=items)*tau
ar =  np.ones(shape=users)*(rho+k*eta)   # not fixed in the original code
br = np.ones(shape=users)*(rho/varrho)   
aw = np.ones(shape=items)*(omega+k*zeta)  # not fixed in the original code
bw = np.ones(shape=items)*(omega/varpi)
varphi = np.zeros([users,items,k])
a_s = np.ones(shape=[users,k])*eta
bs = np.ones(shape=[users,k])*varrho
av = np.ones(shape=[items,k])*zeta
bv = np.ones(shape=[items,k])*varpi

In [ ]:
# curr_loglik = cutoff_loglik -1.0

# while cutoff_loglik > curr_loglik:
#     n_trunc -= 1
#     curr_loglik = n_trunc* np.log(em) - np.log(np.expm1(em)) - math.lgamma(n_trunc + 1);
# n_trunc += 1
# print n_trunc

In [ ]:
param.mle_update(train_non_zero_indices,x,no_samples)

In [ ]:
print param.get_param()

In [ ]:
curr_iter  = 0
while curr_iter <= epochs:
    curr_iter += 1
    u = np.random.randint(low=0,high=users,dtype='int64')
    i = np.random.randint(low=0,high=items,dtype='int64')
    tu = np.power(t_user[u],-xi)
    ti = np.power(t_item[i],-xi)
    
    br[u] = (1.0-tu)*br[u] + tu*(rho/varrho + np.sum(a_s[u,:]/bs[u,:]))
    bs[u,:] = (1.0-tu)*bs[u,:] + tu*(ar[u]/br[u] + items*(av[i,:]/bv[i,:]))
    bw[i] = (1.0-ti)*bw[i] + ti*(omega/varpi + np.sum(av[i,:]/bv[i,:]))
    bv[i,:] = (1.0-tu)*bv[i,:] + ti*(aw[i]/bw[i] + users*(a_s[u,:]/bs[u,:]))
    
    if x[u,i]==0:
        a_s[u,:] = (1.0-tu)*a_s[u,:] + tu*eta
        av[i,:] = (1.0-ti)*av[i,:] + ti*zeta
    else:
        A_ui = np.sum((a_s[u,:]*av[i,:])/(bs[u,:]*bv[i,:]))
        en = param.expectation(x[u,i],A_ui,n_trunc)
        varphi[u,i,:]= sp.digamma(a_s[u,:])-np.log(bs[u,:])+sp.digamma(av[i,:])-np.log(bv[i,:])
        log_norm = logsumexp(varphi[u,i,:])
        varphi[u,i,:] = np.exp(varphi[u,i,:]-log_norm)
        a_s[u,:] = (1.0-tu)*a_s[u,:] + tu*(eta+items*en*varphi[u,i,:])
        av[i,:] = (1.0-ti)*av[i,:] + ti*(zeta+users*en*varphi[u,i,:])
    
    t_user[u] += 1
    t_item[i] += 1
    
    if curr_iter%test_every == 0:
        print curr_iter
        score.append(helper_func.check(a_s,bs,av,bv,test_mask,full_X,metric=metric))
        gc.collect()

In [ ]:
plt.plot(score)
plt.show()
np.savetxt('../results/'+dataset+'/'+'hcpf_po_'+metric+'_'+str(k)+'.txt',np.array(score))
